# Deep Neural Network for MNIST Classification

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'numpy'

## Data

load and preprocess our data.

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

#extract the training and testing dataset with the built references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# there is no validation sets, we will slip from the training dataset
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


# this will take an MNIST image and its label
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


#shuffle the data for better traninig

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# batch the test data
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [6]:
input_size = 784
output_size = 10
hidden_layer_size = 50

#making the neural networl
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
That's where we train the model we have built.

In [8]:
NUM_EPOCHS = 5

# we fit the model, by making sure we are not overfitting the data
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 6s - loss: 0.3206 - accuracy: 0.9105 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 5s - loss: 0.1344 - accuracy: 0.9604 - val_loss: 0.1131 - val_accuracy: 0.9680
Epoch 3/5
540/540 - 5s - loss: 0.0969 - accuracy: 0.9710 - val_loss: 0.0972 - val_accuracy: 0.9745
Epoch 4/5
540/540 - 5s - loss: 0.0752 - accuracy: 0.9776 - val_loss: 0.0841 - val_accuracy: 0.9745
Epoch 5/5
540/540 - 5s - loss: 0.0609 - accuracy: 0.9812 - val_loss: 0.0681 - val_accuracy: 0.9790


## Test the model

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 1s/step - loss: 0.0897 - accuracy: 0.97 - 1s 1s/step - loss: 0.0897 - accuracy: 0.9728

In [10]:
#Finding accuracy of our model 
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.28%
